начало 55.887292, 37.616198  
конец 55.915078, 37.678408


Расчет параметров области

In [1]:
import math

def calculate_area_params(lat1, lon1, lat2, lon2, zoom=18):
    """Рассчитывает параметры области для скачивания"""
    
    # Средняя широта для расчетов
    avg_lat = (lat1 + lat2) / 2
    
    # Размеры области в градусах
    lat_size = abs(lat2 - lat1)  # градусы широты
    lon_size = abs(lon2 - lon1)  # градусы долготы
    
    # Конвертация в километры
    km_per_degree_lat = 111.32
    km_per_degree_lon = 111.32 * math.cos(math.radians(avg_lat))
    
    width_km = lon_size * km_per_degree_lon
    height_km = lat_size * km_per_degree_lat
    
    # Метров в пикселе для зума 18
    meters_per_pixel = 156543.03 * math.cos(math.radians(avg_lat)) / (2 ** zoom)
    
    # Примерный размер в пикселях
    width_pixels = int(width_km * 1000 / meters_per_pixel)
    height_pixels = int(height_km * 1000 / meters_per_pixel)
    
    return {
        'north_west': (max(lat1, lat2), min(lon1, lon2)),  # северо-запад
        'south_east': (min(lat1, lat2), max(lon1, lon2)),  # юго-восток
        'width_km': width_km,
        'height_km': height_km,
        'width_pixels': width_pixels,
        'height_pixels': height_pixels,
        'meters_per_pixel': meters_per_pixel,
        'start_lat': max(lat1, lat2),  # верхний левый угол
        'start_lon': min(lon1, lon2)   # верхний левый угол
    }

# Ваши координаты
start = (55.887292, 37.616198)  # северо-запад
end = (55.915078, 37.678408)    # юго-восток

params = calculate_area_params(start[0], start[1], end[0], end[1])

print("=== ПАРАМЕТРЫ ОБЛАСТИ ===")
print(f"Северо-запад: {params['north_west']}")
print(f"Юго-восток: {params['south_east']}")
print(f"Размер: {params['width_km']:.2f} км x {params['height_km']:.2f} км")
print(f"Примерный размер карты: {params['width_pixels']} x {params['height_pixels']} пикселей")
print(f"Метров в пикселе (зум 18): {params['meters_per_pixel']:.2f}")
print(f"Координаты левого верхнего угла: {params['start_lat']:.6f}, {params['start_lon']:.6f}")

=== ПАРАМЕТРЫ ОБЛАСТИ ===
Северо-запад: (55.915078, 37.616198)
Юго-восток: (55.887292, 37.678408)
Размер: 3.88 км x 3.09 км
Примерный размер карты: 11596 x 9239 пикселей
Метров в пикселе (зум 18): 0.33
Координаты левого верхнего угла: 55.915078, 37.616198


скрипт для скачивания и склейки

In [2]:
import requests
import os
from PIL import Image
import math
import time

class YandexMapDownloader:
    def __init__(self, zoom=18):
        self.zoom = zoom
        
    def latlon_to_tile(self, lat, lon):
        """Конвертирует координаты в номер тайла"""
        lat_rad = math.radians(lat)
        n = 2.0 ** self.zoom
        xtile = int((lon + 180.0) / 360.0 * n)
        ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
        return (xtile, ytile)
    
    def tile_to_latlon(self, x, y):
        """Конвертирует номер тайла в координаты"""
        n = 2.0 ** self.zoom
        lon_deg = x / n * 360.0 - 180.0
        lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * y / n)))
        lat_deg = math.degrees(lat_rad)
        return (lat_deg, lon_deg)
    
    def download_tile(self, x, y):
        """Скачивает один тайл"""
        url = f"https://core-sat.maps.yandex.net/tiles?l=sat&x={x}&y={y}&z={self.zoom}&scale=1&lang=ru_RU"
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Referer': 'https://yandex.ru/maps/'
        }
        
        try:
            response = requests.get(url, headers=headers, timeout=30)
            if response.status_code == 200:
                return response.content
            else:
                print(f"Ошибка {response.status_code} для тайла {x},{y}")
        except Exception as e:
            print(f"Ошибка загрузки тайла {x},{y}: {e}")
        
        return None
    
    def download_area(self, lat1, lon1, lat2, lon2, output_file="big_map.jpg"):
        """Скачивает область и склеивает в одну карту"""
        # Конвертируем координаты в тайлы
        x1, y1 = self.latlon_to_tile(lat1, lon1)
        x2, y2 = self.latlon_to_tile(lat2, lon2)
        
        # Определяем диапазон тайлов
        min_x, max_x = min(x1, x2), max(x1, x2)
        min_y, max_y = min(y1, y2), max(y1, y2)
        
        print(f"Тайлы от ({min_x}, {min_y}) до ({max_x}, {max_y})")
        print(f"Всего тайлов: {(max_x-min_x+1) * (max_y-min_y+1)}")
        
        # Создаем папку для тайлов
        os.makedirs("temp_tiles", exist_ok=True)
        
        # Скачиваем тайлы
        downloaded = 0
        total_tiles = (max_x-min_x+1) * (max_y-min_y+1)
        
        for x in range(min_x, max_x + 1):
            for y in range(min_y, max_y + 1):
                tile_data = self.download_tile(x, y)
                if tile_data:
                    filename = f"temp_tiles/tile_{x}_{y}.jpg"
                    with open(filename, 'wb') as f:
                        f.write(tile_data)
                    downloaded += 1
                
                print(f"Скачано: {downloaded}/{total_tiles} тайлов", end='\r')
                time.sleep(0.1)  # Чтобы не заблокировали
        
        print(f"\nУспешно скачано {downloaded} тайлов")
        
        # Склеиваем тайлы
        if downloaded > 0:
            self._stitch_tiles(min_x, max_x, min_y, max_y, output_file)
            
            # Очищаем временные файлы
            for x in range(min_x, max_x + 1):
                for y in range(min_y, max_y + 1):
                    tile_path = f"temp_tiles/tile_{x}_{y}.jpg"
                    if os.path.exists(tile_path):
                        os.remove(tile_path)
            os.rmdir("temp_tiles")
            
            return lat1, lon1
        else:
            print("Не удалось скачать ни одного тайла!")
            return None, None
    
    def _stitch_tiles(self, min_x, max_x, min_y, max_y, output_file):
        """Склеивает тайлы в одно изображение"""
        tile_size = 256
        width = (max_x - min_x + 1) * tile_size
        height = (max_y - min_y + 1) * tile_size
        
        print(f"Склеиваем карту размером {width}x{height} пикселей...")
        
        result = Image.new('RGB', (width, height))
        
        for x in range(min_x, max_x + 1):
            for y in range(min_y, max_y + 1):
                tile_path = f"temp_tiles/tile_{x}_{y}.jpg"
                if os.path.exists(tile_path):
                    try:
                        tile = Image.open(tile_path)
                        pos_x = (x - min_x) * tile_size
                        pos_y = (y - min_y) * tile_size
                        result.paste(tile, (pos_x, pos_y))
                    except Exception as e:
                        print(f"Ошибка склейки тайла {x},{y}: {e}")
        
        # Сохраняем с максимальным качеством
        result.save(output_file, quality=95, optimize=True)
        print(f"Карта сохранена как: {output_file}")

def main():
    # Ваши координаты
    start_lat, start_lon = 55.887292, 37.616198  # северо-запад
    end_lat, end_lon = 55.915078, 37.678408      # юго-восток
    
    downloader = YandexMapDownloader(zoom=18)
    
    print("Начинаем скачивание карты...")
    start_lat_map, start_lon_map = downloader.download_area(
        start_lat, start_lon, 
        end_lat, end_lon, 
        "my_5km_map.jpg"
    )
    
    if start_lat_map:
        print(f"✅ Карта успешно создана!")
        print(f"📌 Координаты левого верхнего угла: {start_lat_map:.6f}, {start_lon_map:.6f}")
        print(f"📁 Файл: my_5km_map.jpg")
    else:
        print("❌ Ошибка при создании карты")

if __name__ == "__main__":
    main()

Начинаем скачивание карты...
Тайлы от (158463, 81740) до (158508, 81776)
Всего тайлов: 1702
Скачано: 1702/1702 тайлов
Успешно скачано 1702 тайлов
Склеиваем карту размером 11776x9472 пикселей...
Карта сохранена как: my_5km_map.jpg
✅ Карта успешно создана!
📌 Координаты левого верхнего угла: 55.887292, 37.616198
📁 Файл: my_5km_map.jpg


In [7]:
import cv2
import numpy as np
import json
from ultralytics import YOLO
import os
import math

class MapProcessor:
    def __init__(self, model_path, tile_size=640, overlap=0.2):
        """
        Инициализация процессора карт
        """
        print("🔄 Загружаю модель YOLO...")
        self.model = YOLO('Моделюси/yolov8n_gpu_simple_1.pt')
        self.tile_size = tile_size
        self.overlap = overlap
        self.results = []
        
        print(f"✅ Модель загружена. Классы: {list(self.model.names.values())}")
        
    def process_large_map(self, map_path, start_lat, start_lon, meters_per_pixel):
        """
        Обработка большой карты
        
        Args:
            map_path: путь к файлу карты
            start_lat, start_lon: координаты ЛЕВОГО ВЕРХНЕГО угла карты
            meters_per_pixel: метров в одном пикселе карты
        """
        print(f"🗺️ Загружаю карту: {map_path}")
        
        # Загружаем карту
        large_img = cv2.imread(map_path)
        if large_img is None:
            raise Exception(f"❌ Не могу загрузить карту: {map_path}")
            
        h, w = large_img.shape[:2]
        print(f"📏 Размер карты: {w}x{h} пикселей")
        print(f"📍 Левый верхний угол: {start_lat:.6f}, {start_lon:.6f}")
        print(f"📐 Метров в пикселе: {meters_per_pixel:.3f}")
        
        # Разбиваем на тайлы
        tiles = self._split_into_tiles(large_img)
        print(f"🔲 Создано {len(tiles)} тайлов для обработки")
        
        # Обрабатываем каждый тайл
        for i, (tile, tile_x, tile_y) in enumerate(tiles):
            print(f"🔍 Обрабатываю тайл {i+1}/{len(tiles)}...")
            
            # YOLO инференс
            results = self.model(tile, conf=0.3, iou=0.5)
            
            # Обрабатываем результаты
            self._process_tile_results(results, tile_x, tile_y, start_lat, start_lon, meters_per_pixel)
            
            # Прогресс
            if (i + 1) % 10 == 0:
                print(f"📊 Найдено объектов: {len(self.results)}")
        
        print(f"✅ Обработка завершена! Всего найдено объектов: {len(self.results)}")
    
    def _split_into_tiles(self, image):
        """Разбивает изображение на тайлы с перекрытием"""
        h, w = image.shape[:2]
        tiles = []
        stride = int(self.tile_size * (1 - self.overlap))
        
        print(f"🔧 Разбиваю карту с перекрытием {self.overlap*100}%...")
        
        for y in range(0, h, stride):
            for x in range(0, w, stride):
                # Вырезаем тайл
                end_y = min(y + self.tile_size, h)
                end_x = min(x + self.tile_size, w)
                
                tile = image[y:end_y, x:end_x]
                
                # Дополняем до нужного размера если нужно
                if tile.shape[0] < self.tile_size or tile.shape[1] < self.tile_size:
                    tile = cv2.copyMakeBorder(tile, 
                                            0, self.tile_size - tile.shape[0],
                                            0, self.tile_size - tile.shape[1],
                                            cv2.BORDER_CONSTANT, value=[114, 114, 114])
                
                tiles.append((tile, x, y))
        
        return tiles
    
    def _process_tile_results(self, results, tile_x, tile_y, start_lat, start_lon, meters_per_pixel):
        """Обрабатывает результаты одного тайла"""
        for r in results:
            # Обрабатываем сегментацию (полигоны)
            if hasattr(r, 'masks') and r.masks is not None and r.boxes is not None and len(r.boxes.cls) > 0:
                for mask, cls, conf in zip(r.masks.xy, r.boxes.cls, r.boxes.conf):
                    class_name = self.model.names[int(cls)]
                    confidence = float(conf)
                    
                    # Пропускаем дороги как просили
                    if class_name in['road','zrail']:
                        continue
                    
                    # Вычисляем центр полигона
                    center = self._calculate_polygon_center(mask)
                    
                    # Преобразуем в глобальные координаты карты
                    global_x = tile_x + center[0]
                    global_y = tile_y + center[1]
                    
                    # Конвертируем в географические координаты
                    lat, lon = self._pixel_to_geo(global_x, global_y, start_lat, start_lon, meters_per_pixel)
                    
                    # Сохраняем результат
                    self.results.append({
                        'class': class_name,
                        'confidence': round(confidence, 3),
                        'latitude': round(lat, 6),
                        'longitude': round(lon, 6)
                    })
            
    def _calculate_polygon_center(self, polygon):
        """Вычисляет центр полигона"""
        polygon = np.array(polygon)
        center = polygon.mean(axis=0)
        return center
    
    def _pixel_to_geo(self, pixel_x, pixel_y, start_lat, start_lon, meters_per_pixel):
        """
        Конвертирует пиксельные координаты в географические
        """
        # Считаем смещение в метрах от стартовой точки
        meters_east = pixel_x * meters_per_pixel   # смещение на восток
        meters_north = pixel_y * meters_per_pixel  # смещение на север (ОТРИЦАТЕЛЬНОЕ - вниз по карте)
        
        # Конвертируем метры в градусы
        # 1 градус широты ≈ 111.32 км
        lat_offset = -meters_north / 111320  # ОТРИЦАТЕЛЬНОЕ - двигаемся вниз по карте
        # 1 градус долготы зависит от широты
        lon_offset = meters_east / (111320 * math.cos(math.radians(start_lat)))
        
        final_lat = start_lat + lat_offset
        final_lon = start_lon + lon_offset
        
        return final_lat, final_lon
    
    def save_results(self, output_path):
        """Сохраняет результаты в JSON"""
        # Сортируем по классам для удобства
        sorted_results = sorted(self.results, key=lambda x: x['class'])
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(sorted_results, f, indent=2, ensure_ascii=False)
        
        # Статистика по классам
        class_stats = {}
        for item in self.results:
            class_name = item['class']
            class_stats[class_name] = class_stats.get(class_name, 0) + 1
        
        print(f"💾 Результаты сохранены в: {output_path}")
        print("📊 Статистика по классам:")
        for class_name, count in class_stats.items():
            print(f"   {class_name}: {count} объектов")
        
        # Показываем пример
        if sorted_results:
            print("\n📄 Пример объекта:")
            print(json.dumps(sorted_results[0], indent=2, ensure_ascii=False))
    
    def save_results_csv(self, output_path):
        """Сохраняет результаты в CSV для удобства"""
        import csv
        
        with open(output_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['class', 'latitude', 'longitude', 'confidence'])
            
            for result in self.results:
                writer.writerow([
                    result['class'],
                    result['latitude'],
                    result['longitude'],
                    result['confidence']
                ])
        
        print(f"💾 CSV результаты сохранены в: {output_path}")

def calculate_meters_per_pixel(map_width_px, map_height_px, start_lat, start_lon, end_lat, end_lon):
    """Рассчитывает метров в пикселе на основе реальных координат"""
    # Размеры в градусах
    lat_span = abs(start_lat - end_lat)  # градусы широты
    lon_span = abs(start_lon - end_lon)  # градусы долготы
    
    # Конвертация в метры
    avg_lat = (start_lat + end_lat) / 2
    meters_per_degree_lat = 111320  # метров в градусе широты
    meters_per_degree_lon = 111320 * math.cos(math.radians(avg_lat))  # метров в градусе долготы
    
    # Общие размеры в метрах
    height_meters = lat_span * meters_per_degree_lat
    width_meters = lon_span * meters_per_degree_lon
    
    # Метров в пикселе
    meters_per_pixel_x = width_meters / map_width_px
    meters_per_pixel_y = height_meters / map_height_px
    
    # Берем среднее значение
    meters_per_pixel = (meters_per_pixel_x + meters_per_pixel_y) / 2
    
    print(f"📐 Расчет метров в пикселе:")
    print(f"   Размер карты: {map_width_px}x{map_height_px} пикселей")
    print(f"   Реальный размер: {width_meters:.0f}x{height_meters:.0f} метров")
    print(f"   Метров в пикселе: {meters_per_pixel:.3f}")
    
    return meters_per_pixel

def main():
    # === ПРАВИЛЬНЫЕ НАСТРОЙКИ ===
    MODEL_PATH = 'Моделюси/yolov8n_gpu_simple_1.pt'                   # ваша модель YOLO
    MAP_PATH = "my_5km_map.jpg"               # скачанная карта
    OUTPUT_JSON = "detection_results.json"    # выходной файл
    
    # КООРДИНАТЫ ИЗ СКАЧИВАНИЯ:
    START_LAT = 55.887292                     # левый верхний угол (фактический)
    START_LON = 37.616198                     # левый верхний угол (фактический)
    
    # Координаты правого нижнего угла (примерные)
    END_LAT = 55.915078                       # правый нижний угол
    END_LON = 37.678408                       # правый нижний угол
    
    # Размер карты из склейки
    MAP_WIDTH = 11776
    MAP_HEIGHT = 9472
    
    # Рассчитываем точное значение метров в пикселе
    METERS_PER_PIXEL = calculate_meters_per_pixel(
        MAP_WIDTH, MAP_HEIGHT, 
        START_LAT, START_LON, 
        END_LAT, END_LON
    )
    
    # === ЗАПУСК ОБРАБОТКИ ===
    print("\n🚀 Запуск обработки карты YOLO...")
    print("=" * 60)
    print("📐 Параметры области:")
    print(f"   Левый верхний угол: {START_LAT:.6f}, {START_LON:.6f}")
    print(f"   Правый нижний угол: {END_LAT:.6f}, {END_LON:.6f}")
    print(f"   Размер карты: {MAP_WIDTH}x{MAP_HEIGHT} пикселей")
    print(f"   Метров в пикселе: {METERS_PER_PIXEL:.3f}")
    print("=" * 60)
    
    # Создаем процессор
    processor = MapProcessor(
        model_path=MODEL_PATH,
        tile_size=640,
        overlap=0.2
    )
    
    try:
        # Обрабатываем карту
        processor.process_large_map(
            map_path=MAP_PATH,
            start_lat=START_LAT,
            start_lon=START_LON,
            meters_per_pixel=METERS_PER_PIXEL
        )
        
        # Сохраняем результаты в JSON
        processor.save_results(OUTPUT_JSON)
        
        # Сохраняем в CSV (опционально)
        processor.save_results_csv("detection_results.csv")
        
        print("=" * 60)
        print("🎉 Обработка завершена успешно!")
        print("📁 Созданы файлы:")
        print(f"   • {OUTPUT_JSON} - полные результаты в JSON")
        print(f"   • detection_results.csv - результаты в CSV")
        
    except Exception as e:
        print(f"❌ Ошибка: {e}")
        print("\n🔧 Проверьте:")
        print("  1. Файл модели: best.pt")
        print("  2. Файл карты: my_5km_map.jpg") 

if __name__ == "__main__":
    main()

📐 Расчет метров в пикселе:
   Размер карты: 11776x9472 пикселей
   Реальный размер: 3882x3093 метров
   Метров в пикселе: 0.328

🚀 Запуск обработки карты YOLO...
📐 Параметры области:
   Левый верхний угол: 55.887292, 37.616198
   Правый нижний угол: 55.915078, 37.678408
   Размер карты: 11776x9472 пикселей
   Метров в пикселе: 0.328
🔄 Загружаю модель YOLO...
✅ Модель загружена. Классы: ['building', 'field', 'forest', 'lake', 'road', 'zrail']
🗺️ Загружаю карту: my_5km_map.jpg
📏 Размер карты: 11776x9472 пикселей
📍 Левый верхний угол: 55.887292, 37.616198
📐 Метров в пикселе: 0.328
🔧 Разбиваю карту с перекрытием 20.0%...
🔲 Создано 437 тайлов для обработки
🔍 Обрабатываю тайл 1/437...

0: 640x640 6 buildings, 67.1ms
Speed: 8.0ms preprocess, 67.1ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)
🔍 Обрабатываю тайл 2/437...

0: 640x640 2 buildings, 30.0ms
Speed: 3.0ms preprocess, 30.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
🔍 Обрабатываю тайл 3/437..